In [6]:
from datetime import datetime

In [1]:

import mysql.connector
import pandas as pd

def calcular_roi(simbolo, fecha_inicio, fecha_fin):
    # Conexión a la base de datos
    db_connection = mysql.connector.connect(
        host="127.0.0.1",
        user="root",  # Cambia por tu usuario de MySQL
        password="11jablum11",  # Cambia por tu contraseña de MySQL
        database="yfinance"
    )
    cursor = db_connection.cursor()



    # Consulta SQL para obtener el precio inicial y final en el periodo
    query = """
    SELECT fecha, precio_cierre
    FROM precios_historicos
    WHERE id_empresa = (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s)
      AND fecha BETWEEN %s AND %s
    ORDER BY fecha ASC;
    """
    
    cursor.execute(query, (simbolo, fecha_inicio, fecha_fin))
    precios = cursor.fetchall()
    
    # Cerrar conexión
    cursor.close()
    db_connection.close()

    # Verifica si se obtuvieron precios en el rango de fechas
    if len(precios) < 2:
        st.warning("No se encontraron datos suficientes en el rango de fechas seleccionado.")
        return None

    # Precio inicial y final
    precio_inicial = precios[0][1]
    precio_final = precios[-1][1]
    
    # Calcular ROI
    roi = (precio_final - precio_inicial) / precio_inicial * 100

    return roi, pd.DataFrame(precios, columns=['Fecha', 'Precio Cierre'])

### Paso 2: Crear la Interfaz de Streamlit



In [18]:
simbolo = "MMM"  # El símbolo de la acción a consultar
fecha_inicio = datetime(2000, 1, 1)  # Fecha de inicio del periodo
fecha_fin = datetime(2023, 12, 31)   # Fecha de fin del periodo

# Llamar a la función y capturar los resultados
roi, precios_df = calcular_roi(simbolo, fecha_inicio, fecha_fin)

# Verificar y mostrar el resultado
if roi is not None:
    print(f"ROI Total para {simbolo} desde {fecha_inicio.date()} hasta {fecha_fin.date()}: {roi:.2f}%")
    print("\nPrecios en el periodo:")
    print(precios_df)
else:
    print("No se encontraron datos suficientes para calcular el ROI en el rango de fechas seleccionado.")


ROI Total para MMM desde 2000-01-01 hasta 2023-12-31: 348.24%

Precios en el periodo:
           Fecha  Precio Cierre
0     2000-01-03        19.8138
1     2000-01-04        19.0265
2     2000-01-05        19.5776
3     2000-01-06        21.1523
4     2000-01-07        21.5721
...          ...            ...
6032  2023-12-22        86.3833
6033  2023-12-26        87.8293
6034  2023-12-27        88.3412
6035  2023-12-28        89.1130
6036  2023-12-29        88.8124

[6037 rows x 2 columns]
